In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt
from lib.dataset import Dataset
from lib.knn import Knn
from lib.decisionTree import decisionTree

# Part 2

# Analyse supervisé

preparation des données

In [2]:
Data = pd.read_csv("Data/Dataset1.csv")
ds = Dataset(Data)
ds.preprocessData(null="drop",outliers=None,normalisation=None)
Data = ds.data
train=Data.sample(frac=0.8,random_state=42)
test=Data.drop(train.index)

In [3]:
fert2 = test[test["Fertility"] == 2]
fert2.shape

(11, 14)

Définition et Initialisation de la classe 

In [4]:
class Classification:
    #marge is the percentage of the train data between 0 and 1
    def __init__(self, data,marge):
        self.train=data.sample(frac=marge,random_state=42)
        self.test=data.drop(self.train.index)

    def testKnn(self,data=None,distance="euclidienne",k=2):
        
        results = []
        
        self.knn = Knn(np.array(self.train)) 
        if data is None:
            data = self.test

        for i in range(len(data)):
            elem = list(data.iloc[i])[:-1]
            classe = self.knn.getClass(elem,algo=distance,k=k)
            results.append(classe)

        return results
    
    def testDecisionTree(self,data=None,maxDepth=10,minSamplesSplit=2):
        if data is None:
            data = self.test
        
        self.tree = decisionTree(maxDepth=maxDepth,minSamplesSplit=minSamplesSplit)
        trainX,trainY = np.array(self.train)[:,:-1],np.array(self.train)[:,-1].reshape(-1,1)
        self.tree.fit(trainX,trainY)

        testX,testY = np.array(data)[:,:-1],np.array(data)[:,-1].reshape(-1,1)
        pred = self.tree.predict(testX)
        pred = np.array(pred).reshape(-1,1)

        return pred
    
    def testRandomForest(self,data=None):
        pass



    def confMatrix(self,results):
        self.matrix = np.zeros((len(self.test["Fertility"].unique()),len(self.test["Fertility"].unique())))
        for i in range(len(self.test)):
            self.matrix[int(self.test.iloc[i]["Fertility"])][int(results[i])]+=1
        return self.matrix


classificateur = Classification(Data,0.8)

Knn execution

In [5]:
knnResult = classificateur.testKnn()

Tree execution

In [6]:
treeResult = classificateur.testDecisionTree()

Matrice de confusion

In [7]:
classificateur.confMatrix(knnResult)

array([[63.,  7.,  1.],
       [10., 79.,  5.],
       [ 0.,  7.,  4.]])

In [8]:
classificateur.confMatrix(treeResult)

C:\Users\Mekki\AppData\Local\Temp\ipykernel_14128\4248685146.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.matrix[int(self.test.iloc[i]["Fertility"])][int(results[i])]+=1


array([[67.,  4.,  0.],
       [ 9., 81.,  4.],
       [ 0.,  5.,  6.]])